<a href="https://colab.research.google.com/github/yingzi2022/22-Big-Data/blob/main/HW22_s3_rds_us_Beauty_wo_pwd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:8 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:13 http://archive.ubuntu.co

In [ ]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-08-06 12:44:29--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  4.17MB/s    in 0.2s    

2022-08-06 12:44:29 (4.17 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [ ]:
from pyspark import SparkFiles
# Load in selected tsv from S3 into a DataFrame
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Beauty_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

df = spark.read.option('header', 'true').csv(SparkFiles.get("amazon_reviews_us_Beauty_v1_00.tsv.gz"), inferSchema=True, sep='\t', timestampFormat="yyyy/MM/dd")
df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+---------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|          review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+---------------------+-----------+
|         US|    1797882|R3I2DHQBR577SS|B001ANOOOE|       2102612|The Naked Bee Vit...|          Beauty|          5|            0|          0|   N|                Y|          Five Stars| Love this, excell...| 2015-08-31|
|         US|   18381298|R1QNE9NQFJC2Y4|B0016J22EQ|     106393691|Alba Botanica Sun...|          Beauty|          5|

## Drop duplicates and incomplete rows

In [ ]:
print(df.count())
df = df.dropna()
print(df.count())
df = df.dropDuplicates()
print(df.count())

5115666
5114733
5114733


## Examine the schema

In [ ]:
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: string (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



## Rename columns

In [ ]:
df1 = df.withColumnRenamed("marketplace", "country") \
        .withColumnRenamed("customer_id", "customer_id") \
        .withColumnRenamed("review_id", "review_id") \
        .withColumnRenamed("product_id", "product_id") \
        .withColumnRenamed("product_parent", "product_parent") \
        .withColumnRenamed("product_title", "product_title") \
        .withColumnRenamed("product_category", "product_category") \
        .withColumnRenamed("star_rating", "star_rating") \
        .withColumnRenamed("helpful_votes", "helpful_votes") \
        .withColumnRenamed("total_votes", "total_votes") \
        .withColumnRenamed("vine", "vine") \
        .withColumnRenamed("verified_purchase", "verified_purchase") \
        .withColumnRenamed("review_headline", "review_headline") \
        .withColumnRenamed("review_body", "review_body") \
        .withColumnRenamed("review_date", "review_date")
df1.show(5)

+-------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|country|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|     US|   25376189|R1006OY7VTYDQU|B00C224BHC|     925968264|Leegoal(TM) Lovel...|          Beauty|          1|            1|          1|   N|                N|I wouldn't recomm...|It is cheaply mad...| 2014-07-16|
|     US|   22936491|R1009Z54C76YE4|B00GW4FCHE|     232795368|Showgirl Black an...|          Beauty|          5|            1|          

In [ ]:
df1.printSchema()

root
 |-- country: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: string (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



## Create a new DataFrame for review info

In [ ]:
# import data time functions
# create table
from pyspark.sql.functions import *
review_id_table = df1.select(["review_id", "customer_id", "product_id", "product_parent",to_date("review_date", 'yyyy-MM-dd').alias("review_date")])
review_id_table.show(5)


+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R1006OY7VTYDQU|   25376189|B00C224BHC|     925968264| 2014-07-16|
|R1009Z54C76YE4|   22936491|B00GW4FCHE|     232795368| 2014-04-15|
|R100GGYBCRAPXT|   43772801|B001C366FC|     935717996| 2010-04-02|
|R100Q30O1KC9YS|    3672023|B007ZN5ATQ|     240454053| 2014-01-19|
|R100Q4ABWCVLLM|    8208438|B0080CO2F4|     795960598| 2013-07-15|
+--------------+-----------+----------+--------------+-----------+
only showing top 5 rows



In [ ]:
# Check data column type in the table as needed
# review_id_table.printSchema()

In [ ]:
products = df1.select(["product_id", "product_title"]).distinct()
products.show(5)

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B0046KG8DS|Mane 'n Tail 32oz...|
|B0043AV958|Depileve Crystal ...|
|B006IB8QHE|Aura Cacia Rosema...|
|B006G56CP0|Leegoal(TM) House...|
|B000141L58|TIGI Bed Head Hai...|
+----------+--------------------+
only showing top 5 rows



In [ ]:
# Check data column type in the table as needed
# products.printSchema()

In [ ]:
#count customer_id
customers = df1.groupBy('customer_id').agg({"customer_id": "count"})
customers = customers.withColumnRenamed("count(customer_id)", "customer_count")
customers.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|    1327720|             3|
|   39234490|             1|
|   21655184|             1|
|   28862013|             2|
|   12791221|             1|
|   12542214|             6|
|   13867540|            10|
|    8848836|             4|
|    4170434|             1|
|   43876299|             1|
|   11415725|             1|
|   21802892|             1|
|    5904234|             4|
|   10606675|            22|
|   26793187|             1|
|   10477284|             1|
|   31164671|             2|
|   14876455|            18|
|    5450490|             1|
|   23732246|            12|
+-----------+--------------+
only showing top 20 rows



In [ ]:
# Check data column type in the table as needed
# customers.printSchema()

In [ ]:
vine_table = df1.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])
vine_table.show(5)

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R1006OY7VTYDQU|          1|            1|          1|   N|
|R1009Z54C76YE4|          5|            1|          2|   N|
|R100GGYBCRAPXT|          2|            0|          0|   N|
|R100Q30O1KC9YS|          5|            0|          0|   N|
|R100Q4ABWCVLLM|          4|            0|          0|   N|
+--------------+-----------+-------------+-----------+----+
only showing top 5 rows



In [ ]:
# Check data column type in the table as neededvine_table.pr
# intSchema()

## Write DataFrame to RDS

In [ ]:
# Configuration for RDS instance
mode="append"
# jdbc_url = "jdbc:postgresql://<insert endpoint>:5432/my_data_class_db"
jdbc_url = "jdbc:postgresql://<end point>:5432/bigdata_challenge"
config = {"user":"postgres",
          "password": <password>,
          "driver":"org.postgresql.Driver"}

In [ ]:
# Write DataFrame to table created above

review_id_table.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [ ]:
# Write DataFrame to table created above

products.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [ ]:
# Write DataFrame to table created above

customers.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [ ]:
# Write DataFrame to table created above

vine_table.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)